In [122]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
# from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
# import sklearn
import pickle
from keras.applications import *
from keras.preprocessing.image import *
# importing dependencies
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import tqdm ##
from tqdm import * ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

## Take Training Data

In [123]:
folder = 'data/10fold_stacking/' #共同predict對stacling verified data的結果

acc_df = pd.read_csv('data/ens_unverified/validation_ACC_P1S1.csv') #accuracy csv
acc_df.columns = ['model','csv_name','acc']
acc_df = acc_df.filter(['csv_name','acc'])
acc_df['csv_name'] = acc_df['csv_name'].str.replace('_unverified_','_')
files = os.listdir(folder)

ratio_all=0

df_dict = {}
for i,csv in enumerate(files):
    df_name = csv[:csv.rfind('_')]
#     print(df_name)
    if csv.startswith('validation_ACC'):
        continue
    
    ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
    ratio_all += ratio
    if not (df_name in df_dict.keys()):
        df_dict[df_name] = pd.read_csv(os.path.join(folder,csv))
        df1_ = df_dict[df_name].drop('fname',axis=1)
        df1_ = np.array(df1_) * ratio
        df1_ = pd.DataFrame(df1_)
        df1_['acc'] = ratio
#         print(len(df1_))
        df_dict[df_name] = df_dict[df_name].filter(['fname'])
        df_dict[df_name] = pd.merge(df_dict[df_name],df1_,how='inner',right_index=True,left_index=True)
    else:
        df1_name = pd.read_csv(os.path.join(folder,csv))
        df1_ = df1_name.drop('fname',axis=1)
        df1_ = np.array(df1_) * ratio
        df1_ = pd.DataFrame(df1_)
        df1_['acc'] = ratio
#         print(len(df1_))
        df1_name = df1_name.filter(['fname'])
        df1_ = pd.merge(df1_name, df1_,how='inner',right_index=True,left_index=True)
        df_dict[df_name] = df_dict[df_name].append(df1_, ignore_index=True)
#         print(len(df_dict[df_name]))
#     elif csv.startswith('mike_resnet'):
#         ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         if a==0:
#             df1 = pd.read_csv(os.path.join(folder,csv))
#             df1_ = df1.drop('fname',axis=1)
#             df1_ = np.array(df1_) * ratio
#             df1_ = pd.DataFrame(df1_)
#             df1 = df1.filter(['fname'])
#             df1 = pd.merge(df1,df1_,how='inner',right_index=True,left_index=True)
#             a+=1
#         else:
#             df1_name = pd.read_csv(os.path.join(folder,csv))
#             df1_ = df1_name.drop('fname',axis=1)
#             df1_ = np.array(df1_) * ratio
#             df1_ = pd.DataFrame(df1_)
#             df1_name = df1_name.filter(['fname'])
#             df1_ = pd.merge(df1_name, df1_,how='inner',right_index=True,left_index=True)
#             df1.append(df1_, ignore_index=True)
#     elif csv.startswith('mike_cnn2d'):
#         ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         if b==0:
#             df2 = pd.read_csv(os.path.join(folder,csv))
#             df2_ = df2.drop('fname',axis=1)
#             df2_ = np.array(df1_) * ratio
#             df2_ = pd.DataFrame(df1_)
#             df2 = df2.filter(['fname'])
#             df2 = pd.merge(df2,df1_,how='inner',right_index=True,left_index=True)
#             b+=1
#         else:
#             df2.append(pd.read_csv(os.path.join(folder,csv)), ignore_index=True)     
#     elif csv.startswith('mow_cnn2d'):
#         ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         if c==0:
#             df3 = pd.read_csv(os.path.join(folder,csv))
#             c+=1
#         else:
#             df3.append(pd.read_csv(os.path.join(folder,csv)), ignore_index=True)     
    
#     else:
#         print('unknown csv')
#         break
#         ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         print(ratio)
#         ratio_all += ratio
#     df = pd.read_csv(os.path.join(folder,csv),header=None)

for k,v in df_dict.items():
    df_dict[k] = v.sort_values('fname')

_ = list(df_dict.keys())
sum_ = np.zeros((len(df_dict[_[0]]), 42))
for k,v in df_dict.items():
    sum_ += v[v.columns[1:]].values
ratio_ = np.tile(sum_[:, -1], (41, 1)).T
train_X = sum_[:, :-1] / ratio_
#     df_final = pd.DataFrame(sum_)
# df_final

#     sum_ /= ratio_all


# df = pd.merge(df1,df2,on='fname',how='inner')
# df = pd.merge(df,df3,on='fname',how='inner')
# # if df.iloc[0,0] == 'fname':
# df = df.drop('fname',axis=1)
# # df = df.drop(0,axis=1)

# df

# if i==0:
#     train_X = df.values*ratio
# else:
#     train_X += df.values*ratio
train_X.shape

(3710, 41)

## Take Label

In [148]:
label = pd.read_csv('data/train_label.csv',names=['fname','label','verified'],header=0)
dicts_ = pickle.load(open('data/map.pkl','rb'))
label['trans']=label['label'].map(dicts_)
# label = label.drop(['ID','fname','verified','label'],axis=1)
label = pd.merge(label,df_dict[k],on='fname',how='inner')
label = label['trans'].values
print('label like:',label,'data#:', len(label))

label like: [ 1  3  4 ... 14  9 17] data#: 3710


## Split Data to eval

In [125]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid

In [126]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [127]:
# 檢查pure ensemble baseline
sum(np.argmax(train_X,axis=1) == label) / len(label)

0.8455525606469003

In [128]:
## PCA 的話這格別跑
label = to_categorical(label,num_classes=41)
X_train, Y_train, X_valid, Y_valid = split_valid_set(train_X, label, 0.95)
print(X_train.shape , Y_train.shape)

(3524, 41) (3524, 41)


## PCA / Autoencoder降維度

#### PCA

In [109]:
pca = PCA(n_components=32,iterated_power='auto', whiten=True,svd_solver="full",random_state=725035) #n_components='mle',395
train_X_PCA = pca.fit_transform(train_X)
label = to_categorical(label,num_classes=41)
X_train, Y_train, X_valid, Y_valid = split_valid_set(train_X_PCA, label, 0.95)

#### Autoencoder

In [37]:
#DNN autoencoder
input_img = Input(shape=(41,))
x = Dense(32,activation='relu')(input_img) #one_norm #activity
x = Dense(32,activation='relu')(x)
x = Dense(32,activation='relu')(x)
x = Dense(16,activation='relu')(x)
x = Dense(16,activation='relu')(x)
x = Dense(16,activation='relu')(x)
x = Dense(8,activation='relu')(x) #code # 改成256?
encoder = Model(inputs=input_img,outputs=x)
d = Dense(16)(x)
d = Dense(32)(d)
d = Dense(41,activation='sigmoid')(d)
autoencoder = Model(inputs=input_img,outputs=d)
autoencoder.summary()
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 41)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 32)                1344      
_________________________________________________________________
dense_42 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_43 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_44 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_45 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_46 (Dense)             (None, 16)                272       
__________

In [38]:
batchSize=128
patien=15
epoch=300
saveP = 'model/strong_S1P2_autoencoder.h5'
logD = './logs/'+saveP.split('/')[-1].split('.')[0]
opt = Nadam()
autoencoder.compile(optimizer=opt,loss='mse')

history = History()

callback=[
    EarlyStopping(patience=patien,monitor='val_loss',verbose=1),
    ModelCheckpoint(saveP,monitor='val_loss',verbose=1,save_best_only=True, save_weights_only=False),
    TensorBoard(log_dir=logD+'events.epochs'+str(epoch)),
    history,
]
autoencoder.fit(X_train, X_train,
                epochs=epoch,
                batch_size=batchSize,
                shuffle=True,
                validation_data=(X_valid,X_valid),
                callbacks=callback, 
                class_weight='auto'
                )
# model.save(saveP+"_all.h5")
# encoder.save(saveP+'_enc_all.h5')
# encoder.save_weights(saveP+'_enc.h5')

Train on 3524 samples, validate on 186 samples
Epoch 1/300
3524/3524 [==============================] - 0s 76us/step - loss: 0.1325 - val_loss: 0.0149

Epoch 00001: val_loss improved from inf to 0.01490, saving model to model/strong_S1P2_autoencoder.h5
Epoch 2/300
3524/3524 [==============================] - 0s 11us/step - loss: 0.0151 - val_loss: 0.0149

Epoch 00002: val_loss did not improve from 0.01490
Epoch 3/300
3524/3524 [==============================] - 0s 12us/step - loss: 0.0152 - val_loss: 0.0149

Epoch 00003: val_loss improved from 0.01490 to 0.01489, saving model to model/strong_S1P2_autoencoder.h5
Epoch 4/300
3524/3524 [==============================] - 0s 11us/step - loss: 0.0150 - val_loss: 0.0147

Epoch 00004: val_loss improved from 0.01489 to 0.01469, saving model to model/strong_S1P2_autoencoder.h5
Epoch 5/300
3524/3524 [==============================] - 0s 11us/step - loss: 0.0149 - val_loss: 0.0145

Epoch 00005: val_loss improved from 0.01469 to 0.01454, saving mod

In [8]:
model_test = load_model('model/strong_S1P2_autoencoder.h5')
model_test = Model(inputs=model_test.layers[0].input, outputs=model_test.layers[7].output)
model_test.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 41)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 32)                1344      
_________________________________________________________________
dense_42 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_43 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_44 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_45 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_46 (Dense)             (None, 16)                272       
__________

## get latent dim

In [9]:
feature_train = model_test.predict(X_train)
feature_valid = model_test.predict(X_valid)

In [41]:
feature_train.shape

(3524, 8)

## Model

In [12]:
## autoencoder
input_ = Input(shape=(8,))
bn = BatchNormalization()(input_)
dense = Dense(8,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.00005))(bn)
# dense = Dense(8,kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(bn)
# bn = BatchNormalization()(dense)
# dense = LeakyReLU()(dense)

dropout = Dropout(0.005)(dense)

# dense = Dense(164,kernel_initializer='uniform',kernel_regularizer=l2(0.008))(dropout)
bn = BatchNormalization()(dropout)
dense = Dense(41,activation='selu',kernel_initializer='lecun_normal',kernel_regularizer=l2(0.00005))(bn)
# dense = Dense(41,kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(bn)
# bn = BatchNormalization()(dense)
# dense = LeakyReLU()(dense)
dropout = Dropout(0.005)(dense)

bn = BatchNormalization()(dropout)
dense = Dense(41,activation='softmax',kernel_regularizer=l2(0.00005),kernel_initializer='lecun_normal')(bn)
# dense = Dense(41,activation='softmax',kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(dropout)
# dense = LeakyReLU()(dense)
model = Model(inputs=input_, outputs=dense)
# model = Sequential()
# i
# model.add(BatchNormalization())
# model.add(Dense(41,activation='linear',input_shape=(41,)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 8)                 32        
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dropout_3 (Dropout)          (None, 8)                 0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 8)                 32        
_________________________________________________________________
dense_5 (Dense)              (None, 41)                369       
_________________________________________________________________
dropout_4 (Dropout)          (None, 41)                0         
__________

In [13]:
## Autoencoder
batchSize=64
patien=30
epoch=300
saveP = 'model/strong_S1P2_NNclf.h5'
logD = './logs/'+saveP.split('/')[-1].split('.')[0]
opt = Nadam() #Adam(decay=1e-20)#
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['acc'])

history = History()

callback=[
    EarlyStopping(patience=patien,monitor='val_loss',verbose=1),
    ModelCheckpoint(saveP,monitor='val_acc',verbose=1,save_best_only=True, save_weights_only=False),
    TensorBoard(log_dir=logD+'events.epochs'+str(epoch)),
    history,
]
model.fit(feature_train, Y_train,
                epochs=epoch,
                batch_size=batchSize,
                shuffle=True,
                validation_data=(feature_valid,Y_valid),
                callbacks=callback, 
                class_weight='auto'
                )
# model.save(saveP+"_all.h5")
# encoder.save(saveP+'_enc_all.h5')
# encoder.save_weights(saveP+'_enc.h5')

Train on 3524 samples, validate on 186 samples
Epoch 1/300
3524/3524 [==============================] - 1s 196us/step - loss: 3.1725 - acc: 0.1779 - val_loss: 2.6948 - val_acc: 0.2688

Epoch 00001: val_acc improved from -inf to 0.26882, saving model to model/strong_S1P2_NNclf.h5
Epoch 2/300
3524/3524 [==============================] - 0s 30us/step - loss: 2.6168 - acc: 0.2866 - val_loss: 2.5402 - val_acc: 0.2957

Epoch 00002: val_acc improved from 0.26882 to 0.29570, saving model to model/strong_S1P2_NNclf.h5
Epoch 3/300
3524/3524 [==============================] - 0s 29us/step - loss: 2.4857 - acc: 0.3289 - val_loss: 2.4380 - val_acc: 0.3495

Epoch 00003: val_acc improved from 0.29570 to 0.34946, saving model to model/strong_S1P2_NNclf.h5
Epoch 4/300
3524/3524 [==============================] - 0s 32us/step - loss: 2.4055 - acc: 0.3547 - val_loss: 2.3778 - val_acc: 0.3871

Epoch 00004: val_acc improved from 0.34946 to 0.38710, saving model to model/strong_S1P2_NNclf.h5
Epoch 5/300
352

3524/3524 [==============================] - 0s 22us/step - loss: 1.7522 - acc: 0.5502 - val_loss: 1.7740 - val_acc: 0.4839

Epoch 00041: val_acc did not improve from 0.63978
Epoch 42/300
3524/3524 [==============================] - 0s 21us/step - loss: 1.7367 - acc: 0.5522 - val_loss: 1.7570 - val_acc: 0.5699

Epoch 00042: val_acc did not improve from 0.63978
Epoch 43/300
3524/3524 [==============================] - 0s 22us/step - loss: 1.7334 - acc: 0.5553 - val_loss: 1.7495 - val_acc: 0.5430

Epoch 00043: val_acc did not improve from 0.63978
Epoch 44/300
3524/3524 [==============================] - 0s 22us/step - loss: 1.7405 - acc: 0.5485 - val_loss: 1.7637 - val_acc: 0.5806

Epoch 00044: val_acc did not improve from 0.63978
Epoch 45/300
3524/3524 [==============================] - 0s 21us/step - loss: 1.7258 - acc: 0.5647 - val_loss: 1.8726 - val_acc: 0.5215

Epoch 00045: val_acc did not improve from 0.63978
Epoch 46/300
3524/3524 [==============================] - 0s 21us/step - 

3524/3524 [==============================] - 0s 21us/step - loss: 1.5366 - acc: 0.6027 - val_loss: 1.7054 - val_acc: 0.4247

Epoch 00084: val_acc did not improve from 0.69355
Epoch 85/300
3524/3524 [==============================] - 0s 21us/step - loss: 1.5799 - acc: 0.5919 - val_loss: 1.5155 - val_acc: 0.6720

Epoch 00085: val_acc did not improve from 0.69355
Epoch 86/300
3524/3524 [==============================] - 0s 21us/step - loss: 1.5425 - acc: 0.6047 - val_loss: 1.5395 - val_acc: 0.5860

Epoch 00086: val_acc did not improve from 0.69355
Epoch 87/300
3524/3524 [==============================] - 0s 21us/step - loss: 1.5695 - acc: 0.6019 - val_loss: 1.4819 - val_acc: 0.6398

Epoch 00087: val_acc did not improve from 0.69355
Epoch 88/300
3524/3524 [==============================] - 0s 21us/step - loss: 1.5344 - acc: 0.6087 - val_loss: 1.4470 - val_acc: 0.6452

Epoch 00088: val_acc did not improve from 0.69355
Epoch 89/300
3524/3524 [==============================] - 0s 22us/step - 

3524/3524 [==============================] - 0s 21us/step - loss: 1.4521 - acc: 0.6291 - val_loss: 1.6387 - val_acc: 0.5430

Epoch 00127: val_acc did not improve from 0.69355
Epoch 128/300
3524/3524 [==============================] - 0s 21us/step - loss: 1.5205 - acc: 0.6058 - val_loss: 1.6039 - val_acc: 0.5753

Epoch 00128: val_acc did not improve from 0.69355
Epoch 129/300
3524/3524 [==============================] - 0s 21us/step - loss: 1.4740 - acc: 0.6254 - val_loss: 1.3629 - val_acc: 0.6774

Epoch 00129: val_acc did not improve from 0.69355
Epoch 130/300
3524/3524 [==============================] - 0s 22us/step - loss: 1.5096 - acc: 0.6064 - val_loss: 1.4929 - val_acc: 0.5968

Epoch 00130: val_acc did not improve from 0.69355
Epoch 131/300
3524/3524 [==============================] - 0s 22us/step - loss: 1.4674 - acc: 0.6203 - val_loss: 1.2594 - val_acc: 0.7204

Epoch 00131: val_acc improved from 0.69355 to 0.72043, saving model to model/strong_S1P2_NNclf.h5
Epoch 132/300
3524/35

In [119]:
## PCA
input_ = Input(shape=(32,))
bn = BatchNormalization()(input_)
dense = Dense(64,activation='selu',kernel_initializer='lecun_normal')(bn)
# dense = Dense(8,kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(bn)
# bn = BatchNormalization()(dense)
# dense = LeakyReLU()(dense)

dropout = Dropout(0.45)(dense)

# dense = Dense(164,kernel_initializer='uniform',kernel_regularizer=l2(0.008))(dropout)
bn = BatchNormalization()(dropout)
dense = Dense(64,activation='selu',kernel_initializer='lecun_normal')(bn)
# dense = Dense(41,kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(bn)
# bn = BatchNormalization()(dense)
# dense = LeakyReLU()(dense)
dropout = Dropout(0.45)(dense)

bn = BatchNormalization()(dropout)
dense = Dense(41,activation='softmax',kernel_initializer='lecun_normal')(bn)
# dense = Dense(41,activation='softmax',kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(dropout)
# dense = LeakyReLU()(dense)
model = Model(inputs=input_, outputs=dense)
# model = Sequential()
# i
# model.add(BatchNormalization())
# model.add(Dense(41,activation='linear',input_shape=(41,)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 32)                0         
_________________________________________________________________
batch_normalization_64 (Batc (None, 32)                128       
_________________________________________________________________
dense_64 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_43 (Dropout)         (None, 64)                0         
_________________________________________________________________
batch_normalization_65 (Batc (None, 64)                256       
_________________________________________________________________
dense_65 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_44 (Dropout)         (None, 64)                0         
__________

In [121]:
## PCA
batchSize=128
patien=100
epoch=500
saveP = 'model/strong_S1P2_NNclf_PCA.h5'
logD = './logs/'+saveP.split('/')[-1].split('.')[0]
opt = Nadam()#Adamax()#Adam(decay=1e-20)#
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['acc'])

history = History()

callback=[
    EarlyStopping(patience=patien,monitor='val_loss',verbose=1),
    ModelCheckpoint(saveP,monitor='val_acc',verbose=1,save_best_only=True, save_weights_only=False),
    TensorBoard(log_dir=logD+'events.epochs'+str(epoch)),
    history,
]
model.fit(X_train, Y_train,
                epochs=epoch,
                batch_size=batchSize,
                shuffle=True,
                validation_data=(X_valid,Y_valid),
                callbacks=callback, 
                class_weight='auto'
                )
# 0.87062= 32+Nadan/1024 batch

Train on 3524 samples, validate on 186 samples
Epoch 1/500
3524/3524 [==============================] - 1s 262us/step - loss: 0.6867 - acc: 0.8156 - val_loss: 0.6287 - val_acc: 0.8495

Epoch 00001: val_acc improved from -inf to 0.84946, saving model to model/strong_S1P2_NNclf_PCA.h5
Epoch 2/500
3524/3524 [==============================] - 0s 25us/step - loss: 0.6758 - acc: 0.8147 - val_loss: 0.6369 - val_acc: 0.8602

Epoch 00002: val_acc improved from 0.84946 to 0.86022, saving model to model/strong_S1P2_NNclf_PCA.h5
Epoch 3/500
3524/3524 [==============================] - 0s 25us/step - loss: 0.6915 - acc: 0.8170 - val_loss: 0.6290 - val_acc: 0.8495

Epoch 00003: val_acc did not improve from 0.86022
Epoch 4/500
3524/3524 [==============================] - 0s 25us/step - loss: 0.6860 - acc: 0.8138 - val_loss: 0.6302 - val_acc: 0.8441

Epoch 00004: val_acc did not improve from 0.86022
Epoch 5/500
3524/3524 [==============================] - 0s 28us/step - loss: 0.6872 - acc: 0.8116 - va


Epoch 00043: val_acc did not improve from 0.86559
Epoch 44/500
3524/3524 [==============================] - 0s 34us/step - loss: 0.6775 - acc: 0.8212 - val_loss: 0.6452 - val_acc: 0.8441

Epoch 00044: val_acc did not improve from 0.86559
Epoch 45/500
3524/3524 [==============================] - 0s 28us/step - loss: 0.6566 - acc: 0.8170 - val_loss: 0.6496 - val_acc: 0.8602

Epoch 00045: val_acc did not improve from 0.86559
Epoch 46/500
3524/3524 [==============================] - 0s 29us/step - loss: 0.6465 - acc: 0.8246 - val_loss: 0.6445 - val_acc: 0.8548

Epoch 00046: val_acc did not improve from 0.86559
Epoch 47/500
3524/3524 [==============================] - 0s 26us/step - loss: 0.6449 - acc: 0.8269 - val_loss: 0.6431 - val_acc: 0.8710

Epoch 00047: val_acc improved from 0.86559 to 0.87097, saving model to model/strong_S1P2_NNclf_PCA.h5
Epoch 48/500
3524/3524 [==============================] - 0s 27us/step - loss: 0.6437 - acc: 0.8226 - val_loss: 0.6490 - val_acc: 0.8548

Epoch 0

3524/3524 [==============================] - 0s 30us/step - loss: 0.6342 - acc: 0.8275 - val_loss: 0.6429 - val_acc: 0.8548

Epoch 00087: val_acc did not improve from 0.87097
Epoch 88/500
3524/3524 [==============================] - 0s 30us/step - loss: 0.6420 - acc: 0.8297 - val_loss: 0.6474 - val_acc: 0.8387

Epoch 00088: val_acc did not improve from 0.87097
Epoch 89/500
3524/3524 [==============================] - 0s 27us/step - loss: 0.6532 - acc: 0.8246 - val_loss: 0.6484 - val_acc: 0.8333

Epoch 00089: val_acc did not improve from 0.87097
Epoch 90/500
3524/3524 [==============================] - 0s 35us/step - loss: 0.6417 - acc: 0.8292 - val_loss: 0.6506 - val_acc: 0.8495

Epoch 00090: val_acc did not improve from 0.87097
Epoch 91/500
3524/3524 [==============================] - 0s 30us/step - loss: 0.6379 - acc: 0.8243 - val_loss: 0.6570 - val_acc: 0.8495

Epoch 00091: val_acc did not improve from 0.87097
Epoch 92/500
3524/3524 [==============================] - 0s 33us/step - 

#### Original 41-dim

In [138]:

input_ = Input(shape=(41,))
bn = BatchNormalization()(input_)
dense = Dense(82,activation='selu',kernel_initializer='lecun_normal')(bn)
# dense = Dense(8,kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(bn)
# bn = BatchNormalization()(dense)
# dense = LeakyReLU()(dense)

dropout = Dropout(0.43)(dense)

# dense = Dense(164,kernel_initializer='uniform',kernel_regularizer=l2(0.008))(dropout)
bn = BatchNormalization()(dropout)
dense = Dense(82,activation='selu',kernel_initializer='lecun_normal')(bn)
# dense = Dense(41,kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(bn)
# bn = BatchNormalization()(dense)
# dense = LeakyReLU()(dense)
dropout = Dropout(0.43)(dense)

bn = BatchNormalization()(dropout)
dense = Dense(41,activation='softmax',kernel_initializer='lecun_normal')(bn)
# dense = Dense(41,activation='softmax',kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(dropout)
# dense = LeakyReLU()(dense)
model = Model(inputs=input_, outputs=dense)
# model = Sequential()
# i
# model.add(BatchNormalization())
# model.add(Dense(41,activation='linear',input_shape=(41,)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 41)                0         
_________________________________________________________________
batch_normalization_79 (Batc (None, 41)                164       
_________________________________________________________________
dense_79 (Dense)             (None, 82)                3444      
_________________________________________________________________
dropout_53 (Dropout)         (None, 82)                0         
_________________________________________________________________
batch_normalization_80 (Batc (None, 82)                328       
_________________________________________________________________
dense_80 (Dense)             (None, 82)                6806      
_________________________________________________________________
dropout_54 (Dropout)         (None, 82)                0         
__________

In [139]:

batchSize=128
patien=100
epoch=500
saveP = 'model/strong_S1P2_NNclf_41dim_softmax.h5'
logD = './logs/'+saveP.split('/')[-1].split('.')[0]
opt = Adam(decay=1e-20)#
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['acc'])

history = History()

callback=[
    EarlyStopping(patience=patien,monitor='val_loss',verbose=1),
    ModelCheckpoint(saveP,monitor='val_acc',verbose=1,save_best_only=True, save_weights_only=False),
    TensorBoard(log_dir=logD+'events.epochs'+str(epoch)),
    history,
]
model.fit(X_train, Y_train,
                epochs=epoch,
                batch_size=batchSize,
                shuffle=True,
                validation_data=(X_valid,Y_valid),
                callbacks=callback, 
                class_weight='auto'
                )
# 0.87062= 32+Nadan/1024 batch

Train on 3524 samples, validate on 186 samples
Epoch 1/500
3524/3524 [==============================] - 1s 274us/step - loss: 3.3458 - acc: 0.1731 - val_loss: 1.9639 - val_acc: 0.8118

Epoch 00001: val_acc improved from -inf to 0.81183, saving model to model/strong_S1P2_NNclf_41dim_softmax.h5
Epoch 2/500
3524/3524 [==============================] - 0s 31us/step - loss: 2.0548 - acc: 0.5658 - val_loss: 1.0374 - val_acc: 0.8871

Epoch 00002: val_acc improved from 0.81183 to 0.88710, saving model to model/strong_S1P2_NNclf_41dim_softmax.h5
Epoch 3/500
3524/3524 [==============================] - 0s 30us/step - loss: 1.4333 - acc: 0.7517 - val_loss: 0.7270 - val_acc: 0.8925

Epoch 00003: val_acc improved from 0.88710 to 0.89247, saving model to model/strong_S1P2_NNclf_41dim_softmax.h5
Epoch 4/500
3524/3524 [==============================] - 0s 31us/step - loss: 1.1551 - acc: 0.7937 - val_loss: 0.6146 - val_acc: 0.8925

Epoch 00004: val_acc did not improve from 0.89247
Epoch 5/500
3524/3524

3524/3524 [==============================] - 0s 29us/step - loss: 0.6470 - acc: 0.8365 - val_loss: 0.4600 - val_acc: 0.9032

Epoch 00042: val_acc did not improve from 0.91398
Epoch 43/500
3524/3524 [==============================] - 0s 29us/step - loss: 0.6225 - acc: 0.8360 - val_loss: 0.4563 - val_acc: 0.9032

Epoch 00043: val_acc did not improve from 0.91398
Epoch 44/500
3524/3524 [==============================] - 0s 33us/step - loss: 0.6320 - acc: 0.8348 - val_loss: 0.4656 - val_acc: 0.9032

Epoch 00044: val_acc did not improve from 0.91398
Epoch 45/500
3524/3524 [==============================] - 0s 29us/step - loss: 0.6436 - acc: 0.8377 - val_loss: 0.4611 - val_acc: 0.9086

Epoch 00045: val_acc did not improve from 0.91398
Epoch 46/500
3524/3524 [==============================] - 0s 27us/step - loss: 0.6469 - acc: 0.8357 - val_loss: 0.4523 - val_acc: 0.9086

Epoch 00046: val_acc did not improve from 0.91398
Epoch 47/500
3524/3524 [==============================] - 0s 40us/step - 


Epoch 00085: val_acc did not improve from 0.91398
Epoch 86/500
3524/3524 [==============================] - 0s 24us/step - loss: 0.6108 - acc: 0.8314 - val_loss: 0.4688 - val_acc: 0.9032

Epoch 00086: val_acc did not improve from 0.91398
Epoch 87/500
3524/3524 [==============================] - 0s 24us/step - loss: 0.5899 - acc: 0.8470 - val_loss: 0.4709 - val_acc: 0.8978

Epoch 00087: val_acc did not improve from 0.91398
Epoch 88/500
3524/3524 [==============================] - 0s 24us/step - loss: 0.5891 - acc: 0.8439 - val_loss: 0.4683 - val_acc: 0.9032

Epoch 00088: val_acc did not improve from 0.91398
Epoch 89/500
3524/3524 [==============================] - 0s 23us/step - loss: 0.5672 - acc: 0.8502 - val_loss: 0.4669 - val_acc: 0.9032

Epoch 00089: val_acc did not improve from 0.91398
Epoch 90/500
3524/3524 [==============================] - 0s 24us/step - loss: 0.6059 - acc: 0.8405 - val_loss: 0.4750 - val_acc: 0.9032

Epoch 00090: val_acc did not improve from 0.91398
Epoch 91/

3524/3524 [==============================] - 0s 24us/step - loss: 0.5926 - acc: 0.8380 - val_loss: 0.4826 - val_acc: 0.8978

Epoch 00129: val_acc did not improve from 0.91398
Epoch 130/500
3524/3524 [==============================] - 0s 24us/step - loss: 0.5852 - acc: 0.8425 - val_loss: 0.4833 - val_acc: 0.8978

Epoch 00130: val_acc did not improve from 0.91398
Epoch 131/500
3524/3524 [==============================] - 0s 25us/step - loss: 0.5832 - acc: 0.8459 - val_loss: 0.4825 - val_acc: 0.8978

Epoch 00131: val_acc did not improve from 0.91398
Epoch 132/500
3524/3524 [==============================] - 0s 24us/step - loss: 0.5657 - acc: 0.8425 - val_loss: 0.4829 - val_acc: 0.8978

Epoch 00132: val_acc did not improve from 0.91398
Epoch 133/500
3524/3524 [==============================] - 0s 24us/step - loss: 0.5614 - acc: 0.8428 - val_loss: 0.4837 - val_acc: 0.8978

Epoch 00133: val_acc did not improve from 0.91398
Epoch 134/500
3524/3524 [==============================] - 0s 23us/st

In [150]:
model = load_model('model/strong_S1P2_NNclf_PCA.h5')
ans = model.predict(train_X)
ans.shape

ValueError: Error when checking input: expected input_22 to have shape (32,) but got array with shape (41,)

In [147]:
np.argmax(ans,axis=1)
label

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [149]:
sum(np.argmax(ans,axis=1) == label) / len(label)

0.8700808625336928